# Общие методы

## Все библиотеки

In [2]:
import copy
import datetime
import random
import traceback

import numpy as np
import pandas as pd
import seaborn as sns


import torch
from torch import nn, optim
import torch.nn.functional as F
import torcheval.metrics as metrics
from torch.utils.data.dataloader import DataLoader, Dataset  # Классы для простой работы с данными

import skimage.io
from PIL import Image

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets,transforms, models
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torchinfo import summary


import matplotlib.pyplot as plt
import requests


import kagglehub


from IPython.display import display, clear_output

## Фиксация сида рандомайзера

In [3]:
def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True

## Перенос данных на устройство обучения

In [4]:
def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))

## Печать статистику градиентного спуска

In [5]:
def print_grad_stats(model):
    mean = 0
    std = 0
    norm = 1e-5
    for param in model.parameters():
        grad = getattr(param, 'grad', None)
        if grad is not None:
            mean += grad.data.abs().mean()
            std += grad.data.std()
            norm += 1
    mean /= norm
    std /= norm
    print(f'Mean grad {mean}, std {std}, n {norm}')

## Главная функция обучения и валидации

In [6]:
def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0):
    """
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    # Выбор устройства для обучения модели
    if device is None:
        device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    model.to(device)

    # Выбор оптимизатора
    if optimizer_ctor is None:
        # Если не выставлен, исползую Adam
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    else:
        # Иначе используй тот, что выставили
        optimizer = optimizer_ctor(model.parameters(), lr=lr)

    # Выбор регулятора скорости обучения
    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    # Загрузка обучающего датасета
    train_dataloader = data_loader_ctor(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    # Загрузка валидационного датасета
    val_dataloader = data_loader_ctor(val_dataset, batch_size=batch_size, shuffle=False,
                                      num_workers=dataloader_workers_n)

    # Инициализация переменных "лучших" значений потерь и "лучшей" эпохи и модели
    best_val_loss = float('inf')
    best_epoch_i = 0
    best_model = copy.deepcopy(model)

    # По каждой эпохе
    for epoch_i in range(epoch_n):

        try:
            # Сохраняем время начала обучения эпохи
            epoch_start = datetime.datetime.now()
            print('Эпоха {}'.format(epoch_i))

            # Режим обучения модели
            model.train()
            mean_train_loss = 0
            train_batches_n = 0

            # Для каждого батча данных
            for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
                # Если номер батча превысил количество батчей за эпоху, то завершаем
                if batch_i > max_batches_per_epoch_train:
                    break
                
                # Копируем батчи аргументов и ответов на устройство вычисления 
                batch_x = copy_data_to_device(batch_x, device)
                batch_y = copy_data_to_device(batch_y, device)

                # Создаем предсказание
                pred = model(batch_x)

                # Вычисляем отклонение
                loss = criterion(pred, batch_y)

                # Обнуляем градиент
                model.zero_grad()
                loss.backward()

                # Делаем шаг спуска
                optimizer.step()

                # Суммируем значения потери и считаем кол-во батчей
                mean_train_loss += float(loss)
                train_batches_n += 1
            
            # Считаем среднюю потерю за обучение
            mean_train_loss /= train_batches_n

            # Выводим информацию об эпохе

            # print('Эпоха: {} итераций, {:0.2f} сек'.format(train_batches_n,
            #                                                (datetime.datetime.now() - epoch_start).total_seconds()))
            print(f'Эпоха: {train_batches_n} итераций, {(datetime.datetime.now() - epoch_start).total_seconds():0.2f} сек')
            print(f'Среднее значение функции потерь на обучении {mean_train_loss}')

            # Режим валидации модели
            model.eval()
            mean_val_loss = 0
            val_batches_n = 0

            # С отключенным вычислением градиента
            with torch.no_grad():
                # Для каждого батча валидации 
                for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        break
                    
                    # Копируем батчи аргументов и ответов на устройство вычисления                     
                    batch_x = copy_data_to_device(batch_x, device)
                    batch_y = copy_data_to_device(batch_y, device)

                    # Создаем предсказание
                    pred = model(batch_x)
    
                    # Вычисляем отклонение
                    loss = criterion(pred, batch_y)

                    # Суммируем значения потери и считаем кол-во батчей
                    mean_val_loss += float(loss)
                    val_batches_n += 1

            # Считаем среднюю потерю за обучение
            mean_val_loss /= val_batches_n
            print(f'Среднее значение функции потерь на валидации {mean_val_loss}')

            # Если средняя потеря валидации меньше лучшей
            if mean_val_loss < best_val_loss:
                # То сохраняем номер эпохи
                best_epoch_i = epoch_i
                # То значение потери
                best_val_loss = mean_val_loss
                # То сохраняем модель
                best_model = copy.deepcopy(model)
                print('Новая лучшая модель!')
            # Иначе если модель не улучшилась через несколько эпох
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print(f'Модель не улучшилась за последние {early_stopping_patience} эпох, прекращаем обучение')
                # То останвливаем обучение
                break
            # Для существующего регулятора скорости обучения
            if lr_scheduler is not None:
                # Регулируем скорость
                lr_scheduler.step(mean_val_loss)
            print()
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break

    return best_val_loss, best_model


## Вывод предсказания

In [7]:
def predict_with_model(model, dataset, device=None, batch_size=32, num_workers=0, return_labels=False):
    """
    :param model: torch.nn.Module - обученная модель
    :param dataset: torch.utils.data.Dataset - данные для применения модели
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :return: numpy.array размерности len(dataset) x *
    """
    # Определяем устройство
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Инициализируем результаты батчей
    results_by_batch = []

    device = torch.device(device)
    # Загружаем модель на устройство
    model.to(device)
    # И переводим в режим валидации
    model.eval()

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    labels = []

    # С отключенным вычислением градиента
    with torch.no_grad():
        import tqdm
        # Для каждого батча
        for batch_x, batch_y in tqdm.tqdm(dataloader, total=len(dataset)/batch_size):
            
            # Копируем батч на устройство
            batch_x = copy_data_to_device(batch_x, device)

            # Если нужно вернуть названия классов
            if return_labels:
                labels.append(batch_y.numpy())

            # Делаем предсказание
            batch_pred = model(batch_x)

            # Добавляем результаты в массив
            results_by_batch.append(batch_pred.detach().cpu().numpy())
    # Если нужно вернуть названия классов
    if return_labels:
        return np.concatenate(results_by_batch, 0), np.concatenate(labels, 0)
    else:
        return np.concatenate(results_by_batch, 0)

# Класс датасета

## Загрузка датасета из kaggle

In [ ]:
# !pip install kagglehub
# !kaggle config set -n path -v /home/nikosolov/Documents/SamsungMachineLearning/datasets
import kagglehub

# Download latest version
path = kagglehub.dataset_download("datamunge/sign-language-mnist")

print("Path to dataset files:", path)

## Создание трансформера изображения

In [ ]:
img_transforms  = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomVerticalFlip(p=0.25),
    transforms.RandomRotation(degrees=10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[-0.0932, -0.0971, -0.1260], std=[0.5091, 0.4912, 0.4931])
])

## Загрузка изображений

In [ ]:
# Обучающие данные
train_data = torchvision.datasets.ImageFolder(
    root='/kaggle/input/sports-classification/train', 
    transform=img_transforms
)
# Валидационные данные
validation_data = torchvision.datasets.ImageFolder(
    root='/kaggle/input/sports-classification/valid', 
    transform=img_transforms
)
# Тестирующие данные
test_data = torchvision.datasets.ImageFolder(
    root='/kaggle/input/sports-classification/test', 
    transform=img_transforms
)

len(train_data), len(validation_data), len(test_data)

## Загрузка данных с изображений

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    dataset=train_data, 
    batch_size=32, shuffle=True, num_workers=2
)
validation_dataloader = torch.utils.data.DataLoader(
    dataset=validation_data, 
    batch_size=32, shuffle=False, num_workers=2
)
test_dataloader = torch.utils.data.DataLoader(
    dataset=test_data, 
    batch_size=32, shuffle=False, num_workers=2
)

len(train_dataloader), len(validation_dataloader), len(test_dataloader)

In [ ]:
# Создаем класс Датасета
class MNISTdataset(Dataset):
    def __init__(self, file_path):
        # Загрузка датасета из файла по пути
        self.dataset = pd.read_csv(file_path)
        # Создание датасет классов к изображениям
        self.classes_outcome = pd.DataFrame(
            data = np.stack(
                [np.arange(len(self.dataset)), self.dataset.iloc[:, 0].to_numpy()]
                ).T, 
            columns = ["Id", "Class"])
        # Создание трансформера изображения в изображение PIL и в тензор
        self.transform = transforms.Compose(
            [transforms.ToPILImage(), transforms.ToTensor()]
        )
        
    def __len__(self):
        # Получение размера датасета
        return len(self.classes_outcome)

    def __getitem__(self, index):
        # Получение элемента датасета
        label = torch.tensor(int(self.classes_outcome.iloc[index, 1]))
        image = torch.tensor(self.dataset.iloc[index, 1:].to_numpy(), dtype=torch.float32).reshape((1, 28, 28))
        if self.transform:
            image = self.transform(image)
        return [image, label]

# Создание класса модели

## Загрузка предобученной модели

In [ ]:
# Загрузка предобученной модели ResNet
model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

# # Замена последнего слоя
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 100)

# model.to(device)

## Пример сверточной модели

In [ ]:
# Создание классификационной нейронной сети

class CNN(nn.Module):
    # Конструктор класса ( 25 классов букв )
    def __init__(self, num_classes=25):
        super(CNN, self).__init__()
        # Состовляем сверточные слои нейронки
        self.conv_sequential = nn.Sequential(
            # | 1 x 28 x 28
            # Cлой свертки для 1 изображения с размером ядра 3 и паддингом 1 в 32 новых изображения
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            # Слой нормализации значений
            nn.BatchNorm2d(32),
            # Слой функции активации
            nn.ReLU(),
            # Макс пул для уменьшения изображения. Ядро 2 и stride 2, т.е. изображение уменьшится в 2 раза по сторонам
            nn.MaxPool2d(kernel_size=2, stride=2),
            # | 32 x 14 x 14

            # Слои свертки для 32 изображений с размером ядра 3 и паддингом 1 в 64 новых изображения
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            # Слой нормализации
            nn.BatchNorm2d(64),
            # Слой активации
            nn.ReLU(),
            # Слой макс пула для уменьшения изображения. Ядро 2 и stride 2, т.е. изображение уменьшится в 2 раза по сторонам
            nn.MaxPool2d(kernel_size=2, stride=2),
            # | 64 x 7 x 7

            # Слои свертки для 64 изображений с размером ядра 3 и паддингом 1 в 128 новых изображения
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            # Слой нормализации
            nn.BatchNorm2d(128),
            # Слой активации
            nn.ReLU()
            # | 128 x 7 x 7
        )

        # Состовляем линейные слои нейронки
        self.linear_sequential = nn.Sequential(
            # | Регуляризация
            # Зануление половины значений
            nn.Dropout(0.5),
            # | 128 * 7 * 7
            # Слой линейных нейронов 128*7*7 в 512
            nn.Linear(128 * 7 * 7, 512),
            # Слой активации
            nn.ReLU(),
            # | 512
            # Слой линейных нейронов 512 в кол-во классов (25)
            nn.Linear(512, num_classes)
            # | nc
        )

    def forward(self, x):
        x = self.conv_sequential(x)
        # Преобразуем тензор для полносвязного слоя
        x = x.view(-1, 128 * 7 * 7)
        x = self.linear_sequential(x)
        return x

# Обучение модели

In [ ]:
train_eval_loop(
    model=CNN(num_classes=25), 
    train_dataset=train_dataset, 
    val_dataset=test_dataset, 
    criterion=nn.CrossEntropyLoss(),
    lr=1e-4, 
    epoch_n=20, 
    batch_size=2000,
    device="cuda:0" if torch.cuda.is_available() else "cpu", 
    early_stopping_patience=8, 
    l2_reg_alpha=0,
    max_batches_per_epoch_train=10000,
    max_batches_per_epoch_val=1000,
    data_loader_ctor=DataLoader,
    optimizer_ctor=None,
    lr_scheduler_ctor=None,
    shuffle_train=True,
    dataloader_workers_n=0
)

# Тестирование модели

# Сохранение и загрузка модели без прописывания класса модели

In [ ]:
# Сохранение модели
torch.jit.script(model).save('model_scripted.pt')
# Загрузка модели без прописывания класса модели
model = torch.jit.load('model_scripted.pt').eval()

NameError: name 'torch' is not defined